In [2]:
import requests
from tqdm import tqdm
import pandas as pd
import numpy as np
import time
import my_utils as mu
import math
import random
import imp
imp.reload(mu)
tqdm.pandas()

C:\Users\user\AppData\Local\Temp\ipykernel_1612\3360147383.py:9: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [1]:
#티어별 원시 데이터 프레임 생성 함수
def get_rawdata(tier):
    
    lst=[]
    summoner_lst=[]
    key=mu.riot_api_key
    page =random.randrange(1,10)
    division_list=['I','II','III','IV']
    url=''
    
    if (tier =='IRON' or tier =='BRONZE' or tier =='SILVER' or tier =='GOLD' or tier =='PLATINUM' or tier =='DIAMOND'):
        for division in division_list:
            url=f'https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{tier}/{division}?page={page}&api_key={key}'
            res = requests.get(url).json()
            lst+=random.sample(res,5)
    elif tier == 'MASTER':
        url=f'https://kr.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key={key}'
        res = requests.get(url).json()
        lst= random.sample(res['entries'],10)
    elif tier == 'GRANDMASTER':
        url=f'https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key={key}'
        res = requests.get(url).json()
        lst= random.sample(res['entries'],3)
    elif tier == 'CHALLENGER':
        url=f'https://kr.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key={key}'
        res = requests.get(url).json()
        lst= random.sample(res['entries'],1)
    else:
        print('티어를 잘못 입력하셨습니다')
        return
    
    summoner_lst = list(map(lambda x: x['summonerName'],lst))
    df_creates = []
    
    print('원시 데이터 생성중')
    for s in tqdm(summoner_lst):
        try:
            puuid = mu.get_puuid(s)
            if (tier =='IRON' or tier =='BRONZE' or tier =='SILVER' or tier =='GOLD' or tier =='PLATINUM' or tier =='DIAMOND'):
                match_ids = mu.get_match_id(puuid,3)
            elif tier == 'MASTER':
                match_ids = mu.get_match_id_master(puuid,6)
            elif tier == 'GRANDMASTER':
                match_ids = mu.get_match_id_grandmaster(puuid,20)
            elif tier == 'CHALLENGER':
                match_ids = mu.get_match_id_challenger(puuid,60)
            for i in match_ids:
                matches,timeline = mu.get_matches_timelines(i)
                df_creates.append([i,matches,timeline])
        except:
            continue
            
    df = pd.DataFrame(df_creates,columns =['match_id','matches','timeline'])
    return df

In [3]:
#원시 데이터프레임으로 게임 세부내용 데이터프레임 생성 함수
def get_match_timeline_df(raw_data):
    df_creater = []
    info = raw_data.iloc[0].matches['info']['participants']
    print('게임 데이터프레임 생성중...')
    for i in tqdm(range(len(raw_data))):
        try:
            for j in range(len(info)):
                lst = []
                tmp=[]
                a= math.trunc(raw_data.iloc[i].matches['info']['gameDuration']/60)
                lst.append(raw_data.iloc[i]['match_id'])
                lst.append(raw_data.iloc[i].matches['info']['gameDuration'])
                lst.append(raw_data.iloc[i].matches['info']['gameVersion'])
                lst.append(raw_data.iloc[i].matches['info']['participants'][j]['summonerName'])
                lst.append(raw_data.iloc[i].matches['info']['participants'][j]['summonerLevel'])
                lst.append(raw_data.iloc[i].matches['info']['participants'][j]['participantId'])
                lst.append(raw_data.iloc[i].matches['info']['participants'][j]['championName'])
                lst.append(raw_data.iloc[i].matches['info']['participants'][j]['champExperience'])
                lst.append(raw_data.iloc[i].matches['info']['participants'][j]['teamPosition'])
                lst.append(raw_data.iloc[i].matches['info']['participants'][j]['teamId'])
                lst.append(raw_data.iloc[i].matches['info']['participants'][j]['win'])
                lst.append(raw_data.iloc[i].matches['info']['participants'][j]['kills'])
                lst.append(raw_data.iloc[i].matches['info']['participants'][j]['deaths'])
                lst.append(raw_data.iloc[i].matches['info']['participants'][j]['assists'])
                lst.append(raw_data.iloc[i].matches['info']['participants'][j]['totalDamageDealtToChampions'])
                lst.append(raw_data.iloc[i].matches['info']['participants'][j]['totalDamageTaken'])
                if 5<=a: #게임시간이 5분이상일때
                    for k in range(5,a+1):
                        if k<=25: #게임시간 25분까지만 데이터 삽입
                            lst.append(raw_data.iloc[i].timeline['info']['frames'][k]['participantFrames'][str(j+1)]['totalGold'])
                elif a<5: #게임시간이 5분미만일때
                    lst.append(raw_data.iloc[i].timeline['info']['frames'][a]['participantFrames'][str(j+1)]['totalGold'])
                df_creater.append(lst)
        except:continue

    df = pd.DataFrame(df_creater,columns =['gameId','gameDuration','gameVersion','summonerName','summonerLevel','participantId'
                                           ,'championName','champExperience','teamPosition','teamId','win','kills','deaths','assists'
                                           ,'totalDamageDealtToChampions','totalDamageTaken','g_5','g_6','g_7','g_8','g_9','g_10'
                                           ,'g_11','g_12','g_13','g_14','g_15','g_16','g_17','g_18','g_19','g_20','g_21','g_22'
                                           ,'g_23','g_24','g_25'])
    df = df.fillna(0) #NaN값을 0으로
    df=df.astype(dtype='int64',errors='ignore') #데이터 타입을 전부 int로 바꾸고 바꿀 수 없는 것들은 그대로
    df=df.astype({'win':'bool'})
    return df

In [4]:
# mysql 인서트 함수
def insert_my(x,conn):
    query = (
        f'insert into lol_datas (gameId, gameDuration, gameVersion, summonerName, summonerLevel,'
        f'participantId,championName,champExperience,teamPosition,teamId,win,kills,deaths,assists,'
        f'totalDamageDealtToChampions,totalDamageTaken,g_5,g_6,g_7,g_8,g_9,g_10,g_11,g_12,g_13,'
        f'g_14,g_15,g_16,g_17,g_18,g_19,g_20,g_21,g_22,g_23,g_24,g_25)'
        f'values({repr(x.gameId)},{x.gameDuration},{repr(x.gameVersion)},{repr(x.summonerName)},{x.summonerLevel},{x.participantId}'
        f',{repr(x.championName)},{x.champExperience},{repr(x.teamPosition)},{x.teamId},{repr(str(x.win))},{x.kills},{x.deaths},{x.assists},{x.totalDamageDealtToChampions},{x.totalDamageTaken}'
        f',{x.g_5},{x.g_6},{x.g_7},{x.g_8},{x.g_9},{x.g_10},{x.g_11},{x.g_12},{x.g_13},{x.g_14},{x.g_15},{x.g_16},{x.g_17},{x.g_18},{x.g_19},{x.g_20},{x.g_21},{x.g_22},{x.g_23},{x.g_24},{x.g_25})'
    )
    try:
        mu.mysql_execute(query,conn)
    except Exception as e:
        print(e)
    return

In [5]:
# oracle 인서트 함수
def insert_o(x):
    query = (
        f'insert into lol_datas (gameId, gameDuration, gameVersion, summonerName, summonerLevel,'
        f'participantId,championName,champExperience,teamPosition,teamId,win,kills,deaths,assists,'
        f'totalDamageDealtToChampions,totalDamageTaken,g_5,g_6,g_7,g_8,g_9,g_10,g_11,g_12,g_13,'
        f'g_14,g_15,g_16,g_17,g_18,g_19,g_20,g_21,g_22,g_23,g_24,g_25)'
        f'values({repr(x.gameId)},{x.gameDuration},{repr(x.gameVersion)},{repr(x.summonerName)},{x.summonerLevel},{x.participantId}'
        f',{repr(x.championName)},{x.champExperience},{repr(x.teamPosition)},{x.teamId},{repr(str(x.win))},{x.kills},{x.deaths},{x.assists},{x.totalDamageDealtToChampions},{x.totalDamageTaken}'
        f',{x.g_5},{x.g_6},{x.g_7},{x.g_8},{x.g_9},{x.g_10},{x.g_11},{x.g_12},{x.g_13},{x.g_14},{x.g_15},{x.g_16},{x.g_17},{x.g_18},{x.g_19},{x.g_20},{x.g_21},{x.g_22},{x.g_23},{x.g_24},{x.g_25})'
    )
    try:
        mu.oracle_execute(query)
    except Exception as e:
        print(e)
        pass
    return

In [6]:
# mysql 자동 삽입
def auto_insert_my(num):
    tier_lst = ['IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND','MASTER','GRANDMASTER','CHALLENGER']
    for i in range(num):
        tier = random.choices(tier_lst,weights = [0.0209,0.1516, 0.3255, 0.3186, 0.1468, 0.03, 0.0005, 0.0004, 0.0002])[0]
        print('티어:',tier)
        df=get_rawdata(tier)
        tmp=get_match_timeline_df(df)
        conn = mu.connect_mysql('icia')
        print('DB에 데이터 저장중...')
        try:
            tmp.progress_apply(lambda x: insert_my(x,conn),axis =1)
        except Exception as e:
            print(e)
            continue
        conn.commit()
        conn.close()

In [9]:
# 오라클 자동 삽입
def auto_insert_o(num):
    tier_lst = ['IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND','MASTER','GRANDMASTER','CHALLENGER']
    for i in range(num):
        tier = random.choices(tier_lst,weights = [0.0209,0.1516, 0.3255, 0.3186, 0.1468, 0.03, 0.0005, 0.0004, 0.0002])[0]
        print('티어:',tier)
        df=get_rawdata(tier)
        tmp=get_match_timeline_df(df)
        mu.db_open()
        print('DB에 데이터 저장중...')
        try:
            tmp.progress_apply(lambda x: insert_o(x),axis =1)
        except Exception as e:
            print(e)
            continue
        mu.oracle_close()

In [8]:
def insert_matches_timeline_mysql(row,conn):
    # lambda를 이용해서 progress_apply를 통해 insert할 구문 만들기
    query = (
             f'insert into lol_datas(gameId, gameDuration, gameVersion, summonerName, summonerLevel, participantId,'
             f'championName, champExperience, teamPosition, teamId, win, kills, deaths, assists,'
             f'totalDamageDealtToChampions, totalDamageTaken, g_5, g_6, g_7, g_8, g_9, g_10, g_11, g_12 ,g_13,g_14,'
             f'g_15, g_16, g_17, g_18, g_19, g_20, g_21, g_22, g_23, g_24, g_25)'
             f'values(\'{row.gameId}\',{row.gameDuration}, \'{row.gameVersion}\', \'{row.summonerName}\','
             f'{row.summonerLevel}, {row.participantId},\'{row.championName}\',{row.champExperience},'
             f'\'{row.teamPosition}\', {row.teamId}, \'{row.win}\', {row. kills}, {row.deaths}, {row.assists},'
             f'{row.totalDamageDealtToChampions},{row.totalDamageTaken},{row.g_5},{row.g_6},{row.g_7},{row.g_8},'
             f'{row.g_9},{row.g_10},{row.g_11},{row.g_12},{row.g_13},{row.g_14},{row.g_15},{row.g_16},{row.g_17},'
             f'{row.g_18},{row.g_19},{row.g_20},{row.g_21},{row.g_22},{row.g_23},{row.g_24},{row.g_25})'
             f'ON DUPLICATE KEY UPDATE '
             f'gameId = \'{row.gameId}\', gameDuration = {row.gameDuration}, gameVersion = \'{row.gameVersion}\', summonerName= \'{row.summonerName}\','
             f'summonerLevel = {row.summonerLevel},participantId = {row.participantId},championName = \'{row.championName}\','
             f'champExperience = {row.champExperience}, teamPosition = \'{row.teamPosition}\', teamId = {row.teamId},win = \'{row.win}\','
             f'kills = {row. kills}, deaths = {row.deaths}, assists = {row.assists}, totalDamageDealtToChampions = {row.totalDamageDealtToChampions},'
             f'totalDamageTaken = {row.totalDamageTaken},g_5 = {row.g_5},g_6 = {row.g_6},g_7 = {row.g_7},g_8 = {row.g_8},g_9 = {row.g_9},'
             f'g_10 = {row.g_10},g_11 = {row.g_11},g_12 = {row.g_12},g_13 = {row.g_13},g_14 = {row.g_14},g_15 = {row.g_15},g_16 = {row.g_16},g_17 = {row.g_17},'
             f'g_18 = {row.g_18},g_19 = {row.g_19},g_20 = {row.g_20},g_21 = {row.g_21},g_22 = {row.g_22},g_23 = {row.g_23},g_24 = {row.g_24},g_25 = {row.g_25}'
            )
    mu.mysql_execute(query,conn)
    return query

In [ ]:
def insert_matches_timeline(row):
    #데이터프레임에서 apply 한줄씩 넣는다는 전제에서 중복값을 업데이트
    query = (
             f'MERGE INTO lol_datas USING DUAL ON(gameId=\'{row.gameId}\' and participantId={row.participantId}) '
             f'WHEN NOT MATCHED THEN '
             f'insert (gameId, gameDuration, gameVersion, summonerName, summonerLevel, participantId,'
             f'championName, champExperience, teamPosition, teamId, win, kills, deaths, assists,'
             f'totalDamageDealtToChampions, totalDamageTaken, g_5, g_6, g_7, g_8, g_9, g_10, g_11, g_12 ,g_13,g_14,'
             f'g_15, g_16, g_17, g_18, g_19, g_20, g_21, g_22, g_23, g_24, g_25)'
             f'values(\'{row.gameId}\',{row.gameDuration}, \'{row.gameVersion}\', \'{row.summonerName}\','
             f'{row.summonerLevel}, {row.participantId},\'{row.championName}\',{row.champExperience},'
             f'\'{row.teamPosition}\', {row.teamId}, \'{row.win}\', {row. kills}, {row.deaths}, {row.assists},'
             f'{row.totalDamageDealtToChampions},{row.totalDamageTaken},{row.g_5},{row.g_6},{row.g_7},{row.g_8},'
             f'{row.g_9},{row.g_10},{row.g_11},{row.g_12},{row.g_13},{row.g_14},{row.g_15},{row.g_16},{row.g_17},'
             f'{row.g_18},{row.g_19},{row.g_20},{row.g_21},{row.g_22},{row.g_23},{row.g_24},{row.g_25})'
            )
    mu.oracle_execute(query)
    return query

In [ ]:
df=get_rawdata('BRONZE')

In [8]:
# 오라클 테이블 생성 함수
def create_table_o(table_name):
    query = (f'create table {table_name} (gameId varchar(20), gameDuration number(20), gameVersion varchar(20), '
    """
    summonerName varchar(50), summonerLevel number(20), participantId number(20), championName varchar(50), 
    champExperience number(20), teamPosition varchar(20), teamId number(20), win varchar(20), kills number(20), 
    deaths number(20), assists number(20), totalDamageDealtToChampions number(20), totalDamageTaken number(20), g_5 number(20)
    , g_6 number(20), g_7 number(20), g_8 number(20), g_9 number(20), g_10 number(20), g_11 number(20), g_12 number(20), g_13 number(20)
    , g_14 number(20), g_15 number(20), g_16 number(20), g_17 number(20), g_18 number(20), g_19 number(20), g_20 number(20), g_21 number(20)
    , g_22 number(20), g_23 number(20), g_24 number(20), g_25 number(20))
    """)

    mu.db_open()
    mu.oracle_execute(query)
    mu.oracle_close()

In [9]:
# 오라클 테이블 제거 함수
def drop_table_o(table_name):
    query = f'drop table {table_name}'
    mu.db_open()
    mu.oracle_execute(query)
    mu.oracle_close()

In [10]:
# mysql 테이블 생성 함수
def create_table_my(table_name):
    query_mysql = (f'create table {table_name} (gameId varchar(20), gameDuration int, gameVersion varchar(20),' 
    """
    summonerName varchar(50), summonerLevel int, participantId int, championName varchar(50), 
    champExperience int, teamPosition varchar(20), teamId int, win varchar(20), kills int, 
    deaths int, assists int, totalDamageDealtToChampions int, totalDamageTaken int, g_5 int
    , g_6 int, g_7 int, g_8 int, g_9 int, g_10 int, g_11 int, g_12 int, g_13 int
    , g_14 int, g_15 int, g_16 int, g_17 int, g_18 int, g_19 int, g_20 int, g_21 int
    , g_22 int, g_23 int, g_24 int, g_25 int, primary key (gameId, participantId))
    """)
    conn = mu.connect_mysql('icia')
    mu.mysql_execute(query_mysql,conn)
    conn.commit()
    conn.close()

In [11]:
# mysql 테이블 제거 함수
def drop_table_my(table_name):
    query_mysql =f'drop table {table_name}'
    conn = mu.connect_mysql('icia')
    mu.mysql_execute(query_mysql,conn)
    conn.commit()
    conn.close()